In [1]:
# TODO: use `order_no` var

In [2]:
import pandas as pd
from services import get_engine
import graphviz
import numpy as np
import math

pd.options.display.max_columns = None

In [3]:
engine_unf = get_engine(fname='../credentials/.prod_unf')

Connected to bi@prod_unf.


In [4]:
engine_analytics = get_engine(fname='../credentials/.server_analytics')

Connected to Analytics_root@PROD_ANALYTICS.


In [5]:
order_query = '''
-- earned value report.sql
-- get order
SELECT
    order_tbl._LineNo3639 AS orderRowNo
    , order_tbl._Fld35933 AS stage
    , room._Description AS room
    , tech._Description AS tech
    , spec._Description AS spec
    , CAST(spec._IDRRef AS uniqueidentifier) AS specId
    , item._Description AS item
    , CAST(item._IDRRef AS uniqueidentifier) AS itemId
    , order_tbl._Fld3644 AS qty
    , batch._Description AS batch
    , CAST(batch._IDRRef AS uniqueidentifier) AS batchId
FROM 
    _Document164X1 orders
    LEFT JOIN _InfoRg16413 AS numbers ON numbers._Fld16426RRef = orders._IDRRef
    LEFT JOIN _Document164_VT3638X1 AS order_tbl 
        ON order_tbl._Document164_IDRRef = orders._IDRRef
    LEFT JOIN _Reference76 AS item ON item._IDRRef = order_tbl._Fld3640RRef
    LEFT JOIN _Reference79 as tech ON item._Fld1533RRef = tech._IDRRef
    LEFT JOIN _Reference16143 AS room ON room._IDRRef = order_tbl._Fld16181RRef
    LEFT JOIN _Reference89 AS batch ON batch._IDRRef = order_tbl._Fld3643RRef
    LEFT JOIN _Reference16144 AS spec ON order_tbl._Fld16182RRef = spec._IDRRef
WHERE
    numbers._Fld16427 LIKE '13925/13/%'     -- номерЗП
    ORDER BY
        order_tbl._LineNo3639
'''

order = pd.read_sql(order_query, engine_unf)

# Production Orders

In [6]:
# list production orders based on the customer order
prods_query_base = '''
-- earned value report.sql
-- get prod orders    
SELECT CAST(prod._IDRRef AS uniqueidentifier) AS prodId,
    prod._Number AS prodNo,
    status._Description AS prodStatus,
    item._Description AS item,
    batch._Description AS batch,
    CAST(batch._IDRRef AS uniqueidentifier) AS batchId,
    tech.[_Description] AS tech,
    owner._Description AS owner,
    CAST(owner._IDRRef AS uniqueidentifier) AS ownerId,
    parent_prod.[_Number] AS parentProdNo,
    CAST(parent_prod.[_IDRRef] AS uniqueidentifier) AS parentProdId,
    parent_item.[_Description] AS parentItem,
    parent_tech.[_Description] AS parentTech,
    parent_batch.[_Description] AS parentBatch,
    CAST(parent_batch._IDRRef AS uniqueidentifier) AS parentBatchId
FROM _Document163 AS prod
    LEFT JOIN _Reference117 AS status ON prod._Fld3556RRef = status._IDRRef
    LEFT JOIN _Document164X1 AS ordr ON ordr.[_IDRRef] = prod.[_Fld3543RRef]
    LEFT JOIN _InfoRg16413 AS ordr_no ON ordr_no._Fld16426RRef = ordr.[_IDRRef]
    LEFT JOIN _Document163_VT3562 AS prod_tbl ON prod._IDRRef = prod_tbl._Document163_IDRRef
    LEFT JOIN _Reference76 AS item ON item._IDRRef = prod_tbl._Fld3564RRef
    LEFT JOIN _Reference76 AS owner ON owner._IDRRef = prod_tbl._Fld33461RRef
    LEFT JOIN _Reference89 AS batch ON batch._IDRRef = prod_tbl._Fld16259RRef
    LEFT JOIN _Reference79 as tech ON item._Fld1533RRef = tech._IDRRef
    LEFT JOIN _Document163 AS parent_prod ON parent_prod.[_IDRRef] = prod._Fld3555RRef
    LEFT JOIN _Document163_VT3562 AS parent_prod_tbl ON parent_prod._IDRRef = parent_prod_tbl._Document163_IDRRef
    LEFT JOIN _Reference89 AS parent_batch ON parent_batch._IDRRef = parent_prod_tbl._Fld16259RRef
    LEFT JOIN _Reference76 AS parent_item ON parent_item._IDRRef = parent_prod_tbl._Fld3564RRef
    LEFT JOIN _Reference79 as parent_tech ON parent_item._Fld1533RRef = parent_tech._IDRRef
-- need to find all job tickets, not only subordinated to posted prod orders
-- WHERE CAST(prod._Posted AS int) = 1  

'''
prods_query0 = prods_query_base + "WHERE ordr_no._Fld16427 LIKE '13925/13/%'"

prods = pd.read_sql(prods_query0, engine_unf)

In [7]:
# list production orders based on other production orders and not directly based on customer order
def get_all_prods(prods0: pd.DataFrame) -> pd.DataFrame:
    parent_prod_ids = "', '".join(prods0.prodId)
    prods_query1 = prods_query_base + f"""
        -- based on parent prods
        WHERE CAST(parent_prod._IDRRef AS uniqueidentifier) IN ('{parent_prod_ids}')  
        -- not already listed as parent prods. 
        -- TODO: excessive?
        AND CAST(prod._IDRRef AS uniqueidentifier) NOT IN ('{parent_prod_ids}')     
    """
    new_prods = pd.read_sql(prods_query1, engine_unf)
    if len(new_prods) == 0:
        return pd.DataFrame(columns=prods0.columns)

    return pd.concat([new_prods, get_all_prods(new_prods)])

parent_prod_ids = "', '".join(prods.prodId)
prods = pd.concat([prods, get_all_prods(prods)])

In [8]:
prods =prods.merge(
        order[order.batchId.notnull()][['batchId', 'item', 'orderRowNo', 'spec', 'specId']], 
        how='left',
        on=['item', 'batchId'],
        suffixes=('', '_')
)

In [9]:
prods = prods.merge(
        order[order.batchId.notnull()][['batchId', 'item', 'orderRowNo', 'spec', 'specId']], 
        how='left',
        left_on=['parentItem', 'parentBatchId'],
        right_on=['item', 'batchId'],
        suffixes=('', '_parent')
)
prods.drop(columns='batchId_parent', inplace=True)

In [10]:
prods = prods[[
    'prodId',
    'prodNo',
    'prodStatus',
    'item',
    'batch',
    'batchId',
    'tech',
    'orderRowNo',
    'spec',
    'specId',
    'owner',
    'ownerId',
    'parentProdNo',
    'parentProdId',
    'parentItem',
    'parentTech',
    'parentBatch',
    'parentBatchId',
    'item_parent',
    'orderRowNo_parent',
    'specId_parent',
    'spec_parent'
    ]]

In [11]:
# шаблон не должен подчиняться протяжке
parent_cols = [
    'parentProdNo',
    'parentProdId',
    'parentItem',
    'parentBatch',
    'parentBatchId',
    'item_parent',
    'orderRowNo_parent',
    'spec_parent',
    'parentTech'
]

for col in parent_cols:
    prods[col] = prods.apply(
        lambda r: None if r.tech=='Шаблон' and r.parentTech=='Протяжка'
        else r[col] 
        , axis=1
    )

In [12]:
# верхние ЗНП МФР выводятся из под тиража
for col in parent_cols:
    prods[col] = prods.apply(
        lambda r: 
        None if r.tech in ['Модельные работы', 'Формовочные работы'] 
                and r.parentTech=='Фиброгипс'
        else r[col] 
        , axis=1
    )

In [13]:
# shall be empty
# if there is no parent then there shall be a `orderRowNo`
fill_row_nos = prods[(prods.parentItem.isnull()) & (prods.orderRowNo.isnull())]    

In [14]:
# fill missing orderRowNo
for i, row in fill_row_nos.iterrows():
    subset = order[(order.spec==row.owner) & (order.tech==row.tech)]
    if len(subset) != 1:
        print(f'Cannot match order row for prod {row.prodNo} {row["item"]}')
        continue
    prods.at[i, 'orderRowNo'] = subset.orderRowNo.values[0]
    prods.at[i, 'spec'] = subset.spec.values[0]

In [15]:
prods[(prods.parentItem.notnull()) & (prods.orderRowNo.notnull())].tech.unique()    # shall be `Фиброгипс` only

array(['Фиброгипс'], dtype=object)

In [16]:
prod_status_codes = {
    'В очереди на производство': 1,
    'В работе': 2,
    'На стеллажах': 3,
    'Отгружен': 4,
    'Завершен': 5,
    'В ремонте': 1
}

In [17]:
prod_status_codes_reverse = {v: k for k, v in prod_status_codes.items() 
                            if k!='В ремонте'}

In [18]:
prods['prodStatusCode'] = prods.prodStatus.map(prod_status_codes)

# Get budget

Which rate is used to arrive at the price an order is sold?

In [19]:
specs_list = list(order.specId.unique())
specs_list = [spec for spec in specs_list if pd.notnull(spec)]
specs_list = "', '".join(specs_list)
specs_query = f'''
-- complex_specs.sql
SELECT 
    specs._Description AS spec
    , CAST(specs._IDRRef AS uniqueidentifier) AS specId
    , parent._Description AS owner
    , CAST(parent._IDRRef AS uniqueidentifier) AS ownerId
    , parent_tech._Description AS ownerTech
    , CAST(_LineNo16156 AS int) AS specLineNo
    , CAST(component._IDRRef AS uniqueidentifier) AS componentId
    , component._Description AS component
    , component_tech._Description AS componentTech
    , components_list._Fld36159 AS rate
FROM 
    _Reference16144 AS specs
    LEFT JOIN _Reference76 AS parent ON specs._Fld16369RRef = parent._IDRRef
    LEFT JOIN _Reference79 AS parent_tech ON parent._Fld1533RRef = parent_tech._IDRRef
    LEFT JOIN _Reference16144_VT16155 AS components_list 
            ON components_list._Reference16144_IDRRef  = specs._IDRRef
    LEFT JOIN _Reference76 AS component ON components_list._Fld16157_RRRef = component._IDRRef
    LEFT JOIN _Reference79 AS component_tech ON component._Fld1533RRef = component_tech._IDRRef
WHERE 
    CAST(specs.[_IDRRef] AS uniqueidentifier) IN ('{specs_list}')
ORDER BY 
    CAST(specs.[_Code] AS int)
    , CAST(components_list._LineNo16156 AS int)   -- specLineNo
'''

specs = pd.read_sql(specs_query, engine_unf)

In [20]:
def get_rate(row):
    try:
        if pd.isnull(row.spec):
            return specs[row.itemId==specs.componentId].rate.values[0]
        return specs[(row.itemId==specs.componentId) & (row.specId==specs.specId)].rate.values[0]
    except:
        pass

order['rate'] = order.apply(get_rate, axis=1)

In [21]:
order['budget'] = order.rate * order.qty

In [22]:
assert len(order[order.budget.isnull()])==0, 'Not all order row budget calculated.'

# Execution (Job Tickets)

In [23]:
prod_ids = list(prods.prodId.values)
prod_ids = "', '".join(prod_ids)
jobs_query = f'''
-- @job_tickets_tables.sql
-- pulls & castings shops pay monthly
SELECT
    CAST(DATEADD([YEAR], -2000, ticket._Date_Time) AS date) AS jobDate,
    ticket._Number AS job,
    prod_order.[_Number] AS prodOrderNo,
    CAST(prod_order.[_IDRRef] AS uniqueidentifier) AS prodOrderId,
    item._Description AS item,
    CAST(item._IDRRef AS uniqueidentifier) AS itemId,
    owner.[_Description] AS owner,
    CAST(owner._IDRRef AS uniqueidentifier) AS ownerId,
    tech._Description AS tech,
    CAST(tbl._Fld5276RRef AS uniqueidentifier) AS batchId,
    tbl._Fld5271 AS quantity,
    tbl.[_Fld5273] AS rate,
    CAST(tbl._Fld5275 AS float) AS pay
FROM 
    _Document209 ticket 
    LEFT JOIN _Document209_VT5262 tbl ON ticket._IDRRef = tbl._Document209_IDRRef
    LEFT JOIN _Reference76 item ON tbl._Fld5266RRef = item._IDRRef
    LEFT JOIN _Reference79 tech ON tech._IDRRef = item._Fld1533RRef
    LEFT JOIN _Document163 prod_order ON prod_order._IDRRef = ticket._Fld16366RRef
    LEFT JOIN _Document163_VT3562 AS prod_tables 
        ON prod_order._IDRRef = prod_tables._Document163_IDRRef
        AND tbl._LineNo5263 = prod_tables._LineNo3563
    LEFT JOIN _Reference76 owner ON owner.[_IDRRef] = prod_tables._Fld33461RRef 
WHERE 
    CAST(ticket._Posted AS int) = 1
    AND CAST(item._Fld1527 AS int) > 0     -- item_id
    AND CAST(ticket._Fld16366RRef AS uniqueidentifier) IN ('{prod_ids}')
'''
jobs = pd.read_sql(jobs_query, engine_unf)

In [24]:
assert sum(jobs.batchId.isnull()) == 0, 'Job tickets missing batch.'

In [25]:
assert sum(jobs.prodOrderId.isnull()) == 0, 'Production order is missing in Job ticket.'

In [26]:
def get_order_row(prodId):
    prod = prods[prods.prodId==prodId]
    if len(prod) > 0:
        if pd.notnull(prod.orderRowNo.values[0]):
            return prod.orderRowNo.values[0]
        if pd.notnull(prod.orderRowNo_parent.values[0]):
            return prod.orderRowNo_parent.values[0]
        return get_order_row(prod.parentProdId.values[0])

jobs['orderRowNo'] = jobs.prodOrderId.apply(get_order_row)

In [27]:
assert sum(jobs.orderRowNo.isnull())==0, 'Row no not assigned'

# Calc diff

In [28]:
jobs_grouped = jobs.groupby(by='orderRowNo').agg('sum', numeric_only=True).pay

In [29]:
order = order.merge(
    jobs_grouped,
    how='left',
    left_on='orderRowNo',
    right_index=True,
    suffixes=('', '_jobs')
)

In [30]:
assert math.isclose(order.pay.sum(), jobs.pay.sum(), rel_tol=0.001), 'Jobs allocation error.'

# Order Row Min Status

In [31]:
row_min_status = prods[['orderRowNo', 'prodStatusCode']].groupby('orderRowNo').agg(min)

In [32]:
order = order.merge(row_min_status, how='left', 
                    left_on='orderRowNo', right_index=True)
order.rename(columns={'prodStatusCode': 'statusCode'}, inplace=True)

In [33]:
order['status'] = order.statusCode.map(prod_status_codes_reverse)

In [34]:
order.to_sql(
    name='orderBudgetRpt',
    con=engine_analytics,
    if_exists='replace',
    index=False
)

-1

# Output

In [35]:
f = graphviz.Digraph(filename="output.gv")
f.attr(rankdir='LR')  
for _, r in prods.sort_values(by='orderRowNo', ascending=False).iterrows():
    match r.tech:
        case 'Фиброгипс':
            f.attr('node', shape='box')
        case 'Модельные работы':
            f.attr('node', shape='cds')
        case 'Формовочные работы':
            f.attr('node', shape='octagon')
        case 'Шаблон':
            f.attr('node', shape='trapezium')
        case 'Протяжка':
            f.attr('node', shape='house')
        case _:
            f.attr('node', shape='')

    if pd.notnull(r.orderRowNo):
        f.attr('node', style='filled', fillcolor='lightblue')
    else:
        f.attr('node', style='', fillcolor='')
    label = (
            (f'c.{r.orderRowNo:.0f}\n' if pd.notnull(r.orderRowNo) else '') +
            f'{r.prodNo}\n' +
            (f'{r.spec}' if pd.notnull(r.spec) else '---') +
            f'\n{r["item"]}'
    )
    f.node(r.prodNo, label)
for _, r in prods[prods.parentProdNo.notnull()].iterrows():
    if (
            r.parentTech == 'Фиброгипс' 
            and r.tech in ['Модельные работы', 'Формовочные работы']
    ): continue
    f.edge(r.parentProdNo, r.prodNo)

f.render()

'output.gv.pdf'